<a href="https://colab.research.google.com/github/Najemeddinebessaoud/DeepFake_tf/blob/master/Resnext_LSTM_MTCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Restart the runtime
from google.colab import runtime
runtime.unassign()

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [2]:
# Install torch_xla for TPU support
!pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

ERROR: torch_xla-2.0-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


In [3]:
!pip uninstall -y tensorflow
!pip install tensorflow-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00


In [4]:
!pip install torch_xla

In [5]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.4 MB/s eta 0:00:00


In [6]:
!pip uninstall -y lz4 mtcnn
!pip install lz4 mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.1 MB/s eta 0:00:00


In [7]:
import torch_xla.core.xla_model as xm

# Check if TPU is available
device = xm.xla_device()
print(f"Using device: {device}")

Using device: xla:0


In [8]:
import os
import cv2
import glob
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import seaborn as sn
from tqdm import tqdm
import gc
import random
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from sklearn.metrics import confusion_matrix

In [9]:
from mtcnn import MTCNN
# Initialize MTCNN detector
detector = MTCNN()

# Dataset & Preprocessing

In [10]:
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = 1
    frame_count = 0
    while success:
        success, image = vidObj.read()
        if success:
            print(f"Extracted frame {frame_count} from {path}")  # Debug: Print frame count
            yield image
            frame_count += 1
    print(f"Total frames extracted from {path}: {frame_count}")  # Debug: Print total frames

In [11]:
def validate_video(vid_path, train_transforms):
    frames = []
    for i, frame in enumerate(frame_extract(vid_path)):
        # Detect faces in the frame using MTCNN
        faces = detector.detect_faces(frame)

        # Only process frames that contain at least one face
        if len(faces) > 0:
            # Apply transformations to the frame
            transformed_frame = train_transforms(frame)
            frames.append(transformed_frame)

        # Stop after collecting 50 frames with faces
        if len(frames) == 60:
            break

    # Stack the frames into a tensor
    frames = torch.stack(frames)
    return frames

In [12]:
class VideoDataset(Dataset):
    def __init__(self, video_names, labels, sequence_length=10, transform=None, im_size=112):
        self.video_names = video_names
        self.labels = labels
        self.transform = transform
        self.sequence_length = sequence_length
        self.im_size = im_size

    def __len__(self):
        return len(self.video_names)

    def __getitem__(self, idx):
        video_path = self.video_names[idx]
        print(f"Processing video: {video_path}")
        frames = []
        for i, frame in enumerate(self.frame_extract(video_path)):
            if self.transform:
                frame = self.transform(frame)  # Apply transforms
            frames.append(frame)
            if len(frames) == self.sequence_length:
                break

        # Handle cases where no frames are extracted
        if len(frames) == 0:
            print(f"Warning: No frames extracted from {video_path}. Using zero-padded frames.")
            frames = [torch.zeros(3, self.im_size, self.im_size) for _ in range(self.sequence_length)]

        # Pad frames if necessary
        if len(frames) < self.sequence_length:
            padding = [torch.zeros_like(frames[0]) for _ in range(self.sequence_length - len(frames))]
            frames.extend(padding)

        frames = torch.stack(frames)  # Stack frames into a tensor
        label = self.labels[idx]
        return frames, label

    def frame_extract(self, path):
        vidObj = cv2.VideoCapture(path)
        if not vidObj.isOpened():
            print(f"Warning: Unable to open video {path}. Skipping.")
            return
        success = 1
        frame_count = 0
        while success:
            success, image = vidObj.read()
            if success:
                yield image
                frame_count += 1
        print(f"Total frames extracted from {path}: {frame_count}")

In [13]:
def load_videos_and_labels(fake_folder, real_folder):
    fake_videos = glob.glob(os.path.join(fake_folder, '*.mp4'))
    print(f"Found {len(fake_videos)} fake videos in {fake_folder}")

    real_videos = glob.glob(os.path.join(real_folder, '*.mp4'))
    print(f"Found {len(real_videos)} real videos in {real_folder}")

    if len(fake_videos) == 0 and len(real_videos) == 0:
        raise ValueError(f"No videos found in {fake_folder} or {real_folder}. Please check the paths.")

    video_files = fake_videos + real_videos
    labels = [0] * len(fake_videos) + [1] * len(real_videos)

    combined = list(zip(video_files, labels))
    random.shuffle(combined)
    if len(combined) == 0:
        raise ValueError("No videos found after combining fake and real folders.")
    video_files[:], labels[:] = zip(*combined)

    return video_files, labels

In [14]:
import glob
import os
import random

def load_videos_and_labels(fake_folder, real_folder):
    # Find all .mp4 files in the fake folder
    fake_videos = glob.glob(os.path.join(fake_folder, '*.mp4'))
    print(f"Found {len(fake_videos)} fake videos in {fake_folder}")

    # Find all .mp4 files in the real folder
    real_videos = glob.glob(os.path.join(real_folder, '*.mp4'))
    print(f"Found {len(real_videos)} real videos in {real_folder}")

    # Raise an error if no videos are found
    if len(fake_videos) == 0 and len(real_videos) == 0:
        raise ValueError(f"No videos found in {fake_folder} or {real_folder}. Please check the paths.")

    # Combine fake and real video paths
    video_files = fake_videos + real_videos
    # Create labels (0 for fake, 1 for real)
    labels = [0] * len(fake_videos) + [1] * len(real_videos)

    # Shuffle the combined list
    combined = list(zip(video_files, labels))
    random.shuffle(combined)
    if len(combined) == 0:
        raise ValueError("No videos found after combining fake and real folders.")
    video_files[:], labels[:] = zip(*combined)

    return video_files, labels

In [15]:
# Define paths to fake and real video folders
fake_folder = '/content/drive/MyDrive/Colab Notebooks/fake1.1'
real_folder = '/content/drive/MyDrive/Colab Notebooks/real1.1'

# Load video paths and labels
video_files, labels = load_videos_and_labels(fake_folder, real_folder)

# Limit the number of videos to 300 (optional)
video_files = video_files[:40000]
labels = labels[:40000]

# Split into training and validation sets
train_ratio = 0.7
split_idx = int(train_ratio * len(video_files))
train_videos, valid_videos = video_files[:split_idx], video_files[split_idx:]
train_labels, valid_labels = labels[:split_idx], labels[split_idx:]

# Print some info
print(f"Total videos: {len(video_files)}")
print(f"Training videos: {len(train_videos)}")
print(f"Validation videos: {len(valid_videos)}")

Found 17525 fake videos in /content/drive/MyDrive/Colab Notebooks/fake1.1
Found 0 real videos in /content/drive/MyDrive/Colab Notebooks/real1.1
Total videos: 17525
Training videos: 12267
Validation videos: 5258


In [16]:
def oversample_minority_class(video_files, labels):
    # Count the number of fake and real videos
    fake_count = labels.count(0)  # Number of fake videos
    real_count = labels.count(1)  # Number of real videos

    # Find the indices of real videos
    real_indices = [i for i, label in enumerate(labels) if label == 1]

    # Calculate how many real videos need to be duplicated
    oversample_factor = fake_count // real_count  # How many times to duplicate real videos
    oversampled_real_indices = real_indices * oversample_factor  # Duplicate real video indices

    # Add the oversampled real videos to the dataset
    oversampled_video_files = video_files + [video_files[i] for i in oversampled_real_indices]
    oversampled_labels = labels + [labels[i] for i in oversampled_real_indices]

    return oversampled_video_files, oversampled_labels

# Apply oversampling
video_files, labels = oversample_minority_class(video_files, labels)

# Verify the balance
print(f"After oversampling: Fake videos = {labels.count(0)}, Real videos = {labels.count(1)}")

ZeroDivisionError: integer division or modulo by zero

In [ ]:
def oversample_minority_class(video_files, labels):
    # Count the number of fake and real videos
    fake_count = labels.count(0)  # Number of fake videos
    real_count = labels.count(1)  # Number of real videos

    # Find the indices of fake videos
    fake_indices = [i for i, label in enumerate(labels) if label == 0]

    # Calculate how many fake videos need to be duplicated
    oversample_factor = real_count // fake_count
    oversampled_fake_indices = fake_indices * oversample_factor

    # Add the oversampled fake videos to the dataset
    oversampled_video_files = video_files + [video_files[i] for i in oversampled_fake_indices]
    oversampled_labels = labels + [labels[i] for i in oversampled_fake_indices]

    return oversampled_video_files, oversampled_labels

# Apply oversampling
video_files, labels = oversample_minority_class(video_files, labels)

# Verify the balance
print(f"After oversampling: Fake videos = {labels.count(0)}, Real videos = {labels.count(1)}")

In [17]:
import glob
import os
import random
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl

# Load video files and labels
video_files, labels = load_videos_and_labels(fake_folder, real_folder)

# Limit the number of videos to avoid memory overload
video_files = video_files[:40000]  # Adjust this number based on your memory capacity
labels = labels[:40000]

# Apply oversampling to balance the dataset
#video_files, labels = oversample_minority_class(video_files, labels)

# Verify the balance
print(f"After oversampling: Fake videos = {labels.count(0)}, Real videos = {labels.count(1)}")

# Split data into train and validation sets
train_ratio = 0.7
split_idx = int(train_ratio * len(video_files))
train_videos, valid_videos = video_files[:split_idx], video_files[split_idx:]
train_labels, valid_labels = labels[:split_idx], labels[split_idx:]

# Define transforms
im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create datasets
train_data = VideoDataset(train_videos, train_labels, sequence_length=10, transform=train_transforms)
val_data = VideoDataset(valid_videos, valid_labels, sequence_length=10, transform=train_transforms)

# Create DataLoaders with smaller batch size
batch_size = 8  # Reduce batch size to save memory
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Get the TPU device
device = xm.xla_device()
print(f"Using device: {device}")

# Load data onto TPU
train_loader = pl.MpDeviceLoader(train_loader, device)
val_loader = pl.MpDeviceLoader(val_loader, device)

# Calculate class weights
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Define the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)

# Verify class weights
print(f"Class weights: {class_weights}")

Found 17525 fake videos in /content/drive/MyDrive/Colab Notebooks/fake1.1
Found 54786 real videos in /content/drive/MyDrive/Colab Notebooks/real1.1
After oversampling: Fake videos = 9658, Real videos = 30342
Using device: xla:0
Class weights: tensor([1.0354e-04, 3.2958e-05], device='xla:0')


# Checkpoints init

In [18]:
import os

# Define checkpoint directory (save checkpoints to Google Drive for persistence)
checkpoint_dir = "/content/drive/MyDrive/checkpoints1.4"
os.makedirs(checkpoint_dir, exist_ok=True)

In [19]:
def save_checkpoint(epoch, model, optimizer, train_loss, train_accuracy, val_loss, val_accuracy, checkpoint_dir):
    """Save a checkpoint of the model and optimizer."""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy
    }
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    xm.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch} to {checkpoint_path}")

In [20]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """Load a checkpoint to resume training."""
    if os.path.exists(checkpoint_path):
        # Use torch.load to load the checkpoint
        checkpoint = torch.load(checkpoint_path)  # Changed from xm.load to torch.load
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
        print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
        return start_epoch
    else:
        print("No checkpoint found. Starting from scratch.")
        return 1  # Start from epoch 1

# Model & Training

In [21]:
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = torchvision.models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x, None)
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim=1)))

In [22]:
from sklearn.metrics import roc_auc_score
def train_epochs(epoch, num_epochs, train_loader, model, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    para_loader = pl.ParallelLoader(train_loader, [device])  # Use ParallelLoader for TPU
    for batch_idx, (data, target) in enumerate(para_loader.per_device_loader(device)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        _, output = model(data)

        # Debug: Print model output shape
        print(f"Model output shape: {output.shape}")

        loss = criterion(output, target)
        loss.backward()
        xm.optimizer_step(optimizer)  # Use xm.optimizer_step for TPU

        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_samples += target.size(0)
        total_correct += (predicted == target).sum().item()

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    print(f"Epoch [{epoch}/{num_epochs}], Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    return avg_loss, accuracy
from sklearn.metrics import roc_auc_score

def test(epoch, model, valid_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    true_labels = []
    predicted_labels = []
    predicted_probs = []  # Store predicted probabilities

    para_loader = pl.ParallelLoader(valid_loader, [device])  # Use ParallelLoader for TPU
    with torch.no_grad():
        for data, target in para_loader.per_device_loader(device):
            data, target = data.to(device), target.to(device)

            # Forward pass
            _, output = model(data)  # Assuming model returns (features, output)
            loss = criterion(output, target)
            total_loss += loss.item()

            # Get predicted probabilities
            probs = torch.softmax(output, dim=1)  # Apply softmax to get probabilities
            predicted_probs.extend(probs.cpu().numpy())  # Store probabilities

            # Get predicted labels
            _, predicted = torch.max(output.data, 1)
            total_samples += target.size(0)
            total_correct += (predicted == target).sum().item()

            # Store true and predicted labels
            true_labels.extend(target.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(valid_loader)
    accuracy = total_correct / total_samples
    print(f"Validation Epoch [{epoch}], Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    # Calculate ROC-AUC score
    roc_auc = roc_auc_score(true_labels, np.array(predicted_probs)[:, 1])  # Use probabilities for class 1
    print(f"Validation Epoch [{epoch}], ROC-AUC: {roc_auc:.4f}")

    return true_labels, predicted_labels, predicted_probs, avg_loss, accuracy, roc_auc

In [23]:
def train_fn(rank):
    device = xm.xla_device()
    print(f"Using device: {device}")

    # Define checkpoint directory
    checkpoint_dir = "/content/drive/MyDrive/checkpoints1.4"
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Create DataLoaders with smaller batch size
    batch_size = 4  # Reduce batch size to save memory
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
    valid_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=0)

    # Define model, optimizer, and loss function
    model = Model(2).to(device)
    lr = 1e-5
    num_epochs = 100
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss().to(device)

    # Load checkpoint if it exists
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_epoch_62.pth")  # Example path
    start_epoch = load_checkpoint(checkpoint_path, model, optimizer)

    # Lists to store metrics
    train_loss_avg = []
    train_accuracy = []
    test_loss_avg = []
    test_accuracy = []
    best_accuracy = 0

    # Training loop
    for epoch in range(start_epoch, num_epochs + 1):
        print(f"Starting Epoch {epoch}")

        # Train for one epoch
        l, acc = train_epochs(epoch, num_epochs, train_loader, model, criterion, optimizer, device)
        train_loss_avg.append(l)
        train_accuracy.append(acc)

        # Validate after each epoch
        true, pred, predicted_probs, tl, t_acc, roc_auc = test(epoch, model, valid_loader, criterion, device)
        test_loss_avg.append(tl)
        test_accuracy.append(t_acc)

        # Save checkpoint every 5 epochs
        if epoch % 2 == 0:
            save_checkpoint(epoch, model, optimizer, l, acc, tl, t_acc, checkpoint_dir)

        # Save the best model based on validation accuracy
        if t_acc > best_accuracy:
            best_accuracy = t_acc
            best_model_path = os.path.join(checkpoint_dir, "best_model.pth")
            xm.save(model.state_dict(), best_model_path)
            print(f"Best model saved at epoch {epoch} with validation accuracy {t_acc:.4f}")

    # After training completes, save the final model and training metrics
    final_model_path = os.path.join(checkpoint_dir, 'final_model.pth')
    final_metrics_path = os.path.join(checkpoint_dir, 'training_metrics.pth')

    # Save final model
    xm.save(model.state_dict(), final_model_path)

    # Save comprehensive training information
    torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss_history': train_loss_avg,
        'train_accuracy_history': train_accuracy,
        'val_loss_history': test_loss_avg,
        'val_accuracy_history': test_accuracy,
        'best_accuracy': best_accuracy,
        'roc_auc': roc_auc,
        'true_labels': true,
        'predicted_labels': pred,
        'predicted_probs': predicted_probs
    }, final_metrics_path)

    print(f"Final model saved to {final_model_path}")
    print(f"Training metrics saved to {final_metrics_path}")

    # Plot results and confusion matrix
    if rank == 0:
        plot_loss(train_loss_avg, test_loss_avg, len(train_loss_avg))
        plot_accuracy(train_accuracy, test_accuracy, len(train_accuracy))
        print_confusion_matrix(true, pred)

        # Plot ROC curve if needed
        fpr, tpr, thresholds = roc_curve(true, np.array(predicted_probs)[:, 1])
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.4f})")
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Random Guess")
        plt.xlabel("False Positive Rate (FPR)")
        plt.ylabel("True Positive Rate (TPR)")
        plt.title("ROC Curve")
        plt.legend()
        plt.show()

# Plots & Metrics

In [24]:
def plot_loss(train_loss, test_loss, epochs):
    plt.plot(range(1, epochs + 1), train_loss, label='Train Loss')
    plt.plot(range(1, epochs + 1), test_loss, label='Validation Loss')
    plt.title('Loss vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_accuracy(train_accuracy, test_accuracy, epochs):
    plt.plot(range(1, epochs + 1), train_accuracy, label='Train Accuracy')
    plt.plot(range(1, epochs + 1), test_accuracy, label='Validation Accuracy')
    plt.title('Accuracy vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def print_confusion_matrix(true, pred):
    cm = confusion_matrix(true, pred)
    plt.figure(figsize=(10, 7))
    sn.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.show()

In [25]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Assume you want to test after the last epoch of training
# Replace this with the actual number of epochs you trained for
num_epochs = 100

# After training, validate the model
true_labels, predicted_labels, predicted_probs, val_loss, val_accuracy, roc_auc = test(
    num_epochs, model, val_loader, criterion, device # Pass num_epochs instead of epoch
)

# Print ROC-AUC score
print(f"Final ROC-AUC Score: {roc_auc:.4f}")

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(true_labels, np.array(predicted_probs)[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Random Guess")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve")
plt.legend()
plt.show()

NameError: name 'model' is not defined

# Main

In [26]:
xmp.spawn(train_fn, nprocs=1)

Using device: xla:0


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 222MB/s]


Loaded checkpoint from epoch 62
Starting Epoch 63
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/W015_light_uniform_angry_camera_left.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/fake1.1/aug_1_115_M123.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/M139_light_right_happy_camera_leftfront.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/fake1.1/aug_1_308_W101_1.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/W036_light_rightdown_surprise_camera_right.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/M024_light_up_happy_camera_left.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/fake1.1/aug_1_772_W028_1.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/aug_0_W133_light_down_disgust_camera_leftfront.mp4
Processing video: /content/drive/MyDrive/Colab Notebooks/real1.1/M117_light_rightup_neutral_camera_right.mp4
Processing video: /content/drive/MyDrive/

KeyboardInterrupt: 

# Test

In [49]:
import torch
import torch.nn as nn
from torchvision import transforms, models
import torch_xla
import torch_xla.core.xla_model as xm
import cv2
import numpy as np
from PIL import Image
from google.colab import files

# 1. Define the EXACT same model class used during training
class VideoModel(nn.Module):
    def __init__(self, num_classes=2):
        super(VideoModel, self).__init__()
        # Use the same backbone name as during training
        backbone = models.resnext50_32x4d(weights=None)
        self.model = nn.Sequential(*list(backbone.children())[:-2])  # Note: using 'model' not 'backbone'
        self.lstm = nn.LSTM(2048, 2048, 1, bidirectional=False)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x)
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim=1)))

# 2. Initialize TPU
device = xm.xla_device()
print(f"Using device: {device}")

# 3. Load model with proper key mapping
model = VideoModel().to(device)

# Load checkpoint
checkpoint = torch.load('/content/drive/MyDrive/model testing/92ACC.pth',
                       map_location='cpu')

# Fix key mismatches by removing 'module.' prefix if present
state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint
new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace('module.', '')  # Remove 'module.' if using DataParallel
    new_state_dict[name] = v

model.load_state_dict(new_state_dict, strict=False)  # strict=False to ignore non-matching keys
model.eval()

# 4. Define transforms (must match training)
transform = transforms.Compose([
    transforms.Lambda(lambda x: Image.fromarray(x) if isinstance(x, np.ndarray) else x),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 5. Prediction function with TPU support
def predict_video(video_path, max_frames=2000000):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened() and len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(transform(frame))

    if not frames:
        raise ValueError("No frames could be read from the video")

    frames = torch.stack(frames).unsqueeze(0).to(device)

    with torch.no_grad():
        _, output = model(frames)
        proba = torch.softmax(output, dim=1)
        pred = proba.argmax().item()
        confidence = proba[0][pred].item()

    return "Real" if pred == 1 else "Fake", confidence

# 6. Test with video upload
print("Please upload a video file...")
uploaded = files.upload()
video_file = next(iter(uploaded))
prediction, confidence = predict_video(video_file)
print(f"Prediction: {prediction} (Confidence: {confidence:.2%})")

Using device: xla:0
Please upload a video file...


Saving 0ABCBZ0CWN.jpg to 0ABCBZ0CWN.jpg
Prediction: Real (Confidence: 100.00%)


In [50]:
import torch
import torch.nn as nn
from torchvision import transforms, models
import torch_xla
import torch_xla.core.xla_model as xm
import cv2
import numpy as np
from PIL import Image
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from google.colab import drive
from tqdm import tqdm

# Mount Google Drive if not already mounted
try:
    drive.mount('/content/drive', force_remount=False)
except:
    print("Drive already mounted")

# Define the model class exactly as trained
class VideoModel(nn.Module):
    def __init__(self, num_classes=2):
        super(VideoModel, self).__init__()
        backbone = models.resnext50_32x4d(weights=None)
        self.model = nn.Sequential(*list(backbone.children())[:-2])
        self.lstm = nn.LSTM(2048, 2048, 1, bidirectional=False)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x)
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim=1)))

# Initialize TPU
print("Initializing TPU...")
try:
    device = xm.xla_device()
    print(f"Using device: {device}")
except:
    device = torch.device("cpu")
    print(f"TPU initialization failed, falling back to: {device}")

# Define preprocessing transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: Image.fromarray(x) if isinstance(x, np.ndarray) else x),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_model(model_path):
    print(f"Loading model from: {model_path}")
    model = VideoModel()

    # Load checkpoint
    try:
        checkpoint = torch.load(model_path, map_location='cpu')

        # Fix key mismatches by removing 'module.' prefix if present
        state_dict = checkpoint['state_dict'] if isinstance(checkpoint, dict) and 'state_dict' in checkpoint else checkpoint
        new_state_dict = {}
        for k, v in state_dict.items():
            name = k.replace('module.', '')  # Remove 'module.' if using DataParallel
            new_state_dict[name] = v

        model.load_state_dict(new_state_dict, strict=False)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

    model = model.to(device)
    model.eval()
    return model

def extract_frames(video_path, max_frames=100, uniform_sampling=True):
    """Extract frames with intelligent sampling for better representation"""
    frames = []

    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Warning: Could not open {video_path}")
            return frames

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if total_frames <= 0:
            print(f"Warning: Invalid frame count for {video_path}")
            return frames

        # Select frames based on video length
        if uniform_sampling and total_frames > max_frames:
            # Sample frames uniformly across the video
            indices = np.linspace(0, total_frames-1, max_frames, dtype=int)
        else:
            # Take sequential frames up to max_frames
            indices = range(min(total_frames, max_frames))

        # Extract the frames
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                # Convert to RGB and apply transform
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_tensor = transform(frame)
                frames.append(frame_tensor)

        cap.release()

    except Exception as e:
        print(f"Error extracting frames from {video_path}: {e}")

    return frames

def predict_video_with_batching(video_path, model, frame_batch_size=32):
    """Process video with memory-efficient frame batching"""
    try:
        # Extract frames
        frames = extract_frames(video_path)

        if not frames:
            return "Error", 0.0, "No valid frames extracted"

        num_frames = len(frames)

        # Process frames in smaller batches to manage memory
        predictions = []
        confidences = []

        # Split frames into manageable batches
        for i in range(0, num_frames, frame_batch_size):
            end_idx = min(i + frame_batch_size, num_frames)
            batch_frames = frames[i:end_idx]

            # Process this batch
            batch_tensor = torch.stack(batch_frames).unsqueeze(0).to(device)

            with torch.no_grad():
                _, output = model(batch_tensor)
                proba = torch.softmax(output, dim=1)

                # Store results from this batch
                batch_pred = proba.argmax(dim=1).item()
                batch_conf = proba[0][batch_pred].item()

                predictions.append(batch_pred)
                confidences.append(batch_conf)

        # Combine batch results (majority voting)
        final_pred = 1 if sum(pred == 1 for pred in predictions) > len(predictions) / 2 else 0
        avg_conf = sum(confidences) / len(confidences)

        result = "Real" if final_pred == 1 else "Fake"
        return result, avg_conf, f"Processed {num_frames} frames"

    except Exception as e:
        return "Error", 0.0, str(e)

def batch_test_videos(real_dir, fake_dir, model, results_csv="model_evaluation_results.csv"):
    """Test all videos in directories and produce evaluation metrics"""
    print(f"Starting batch testing from directories:\nReal: {real_dir}\nFake: {fake_dir}")

    # Check if directories exist
    if not os.path.exists(real_dir):
        print(f"Error: Directory {real_dir} not found")
        return None
    if not os.path.exists(fake_dir):
        print(f"Error: Directory {fake_dir} not found")
        return None

    results = []
    true_labels = []
    pred_labels = []
    confidences = []
    file_paths = []

    # Process real videos
    if os.path.exists(real_dir):
        print("Processing REAL videos...")
        for filename in tqdm(os.listdir(real_dir)):
            if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                file_path = os.path.join(real_dir, filename)
                prediction, confidence, status = predict_video_with_batching(file_path, model)

                results.append({
                    'filename': filename,
                    'true_class': 'Real',
                    'predicted_class': prediction,
                    'confidence': confidence,
                    'status': status
                })

                true_labels.append(1)  # 1 for real
                pred_labels.append(1 if prediction == "Real" else 0)
                confidences.append(confidence)
                file_paths.append(file_path)

    # Process fake videos
    if os.path.exists(fake_dir):
        print("Processing FAKE videos...")
        for filename in tqdm(os.listdir(fake_dir)):
            if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                file_path = os.path.join(fake_dir, filename)
                prediction, confidence, status = predict_video_with_batching(file_path, model)

                results.append({
                    'filename': filename,
                    'true_class': 'Fake',
                    'predicted_class': prediction,
                    'confidence': confidence,
                    'status': status
                })

                true_labels.append(0)  # 0 for fake
                pred_labels.append(1 if prediction == "Real" else 0)
                confidences.append(confidence)
                file_paths.append(file_path)

    # Save detailed results to CSV
    df = pd.DataFrame(results)
    df.to_csv(results_csv, index=False)
    print(f"Detailed results saved to {results_csv}")

    # Return data for metrics calculation
    return true_labels, pred_labels, confidences, file_paths, df

def calculate_metrics(true_labels, pred_labels, confidences, file_paths, results_df):
    """Calculate and visualize performance metrics"""
    if not true_labels or len(true_labels) != len(pred_labels):
        print("Error: Invalid or empty label data")
        return

    # Convert to numpy arrays for analysis
    y_true = np.array(true_labels)
    y_pred = np.array(pred_labels)
    y_scores = np.array(confidences)

    # 1. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Fake', 'Real'],
                yticklabels=['Fake', 'Real'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

    # 2. Classification Report
    report = classification_report(y_true, y_pred, target_names=['Fake', 'Real'], output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    print("\nClassification Report:")
    print(report_df)
    report_df.to_csv('classification_report.csv')

    # 3. ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('roc_curve.png')
    plt.close()

    # 4. Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(10, 8))
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.tight_layout()
    plt.savefig('precision_recall_curve.png')
    plt.close()

    # 5. Analysis of Misclassifications
    misclassified = results_df[results_df['true_class'] != results_df['predicted_class']]
    print("\nSample of Misclassified Videos:")
    print(misclassified.head(10))

    # 6. Confidence Distribution by True Class
    plt.figure(figsize=(12, 6))
    sns.histplot(data=results_df, x='confidence', hue='true_class',
                 element='step', stat='density', common_norm=False, bins=20)
    plt.title('Confidence Score Distribution by True Class')
    plt.xlabel('Confidence Score')
    plt.ylabel('Density')
    plt.tight_layout()
    plt.savefig('confidence_distribution.png')
    plt.close()

    # 7. Confidence Threshold Analysis
    thresholds = np.linspace(0, 1, 100)
    accuracy_scores = []

    for threshold in thresholds:
        # Adjust predictions based on confidence threshold
        adjusted_preds = [(1 if confidence >= threshold else 0) if pred == 1 else
                         (0 if confidence >= threshold else 1)
                         for pred, confidence in zip(y_pred, y_scores)]

        # Calculate accuracy at this threshold
        accuracy = np.mean(np.array(adjusted_preds) == y_true)
        accuracy_scores.append(accuracy)

    # Find optimal threshold
    optimal_idx = np.argmax(accuracy_scores)
    optimal_threshold = thresholds[optimal_idx]
    optimal_accuracy = accuracy_scores[optimal_idx]

    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, accuracy_scores, 'b-')
    plt.axvline(x=optimal_threshold, color='r', linestyle='--',
               label=f'Optimal threshold: {optimal_threshold:.2f}\nAccuracy: {optimal_accuracy:.2f}')
    plt.xlabel('Confidence Threshold')
    plt.ylabel('Accuracy')
    plt.title('Impact of Confidence Threshold on Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('threshold_analysis.png')
    plt.close()

    # 8. Summary table
    tn, fp, fn, tp = cm.ravel()
    summary = {
        'Total Videos': len(y_true),
        'Correct Predictions': np.sum(y_true == y_pred),
        'True Positive': tp,
        'True Negative': tn,
        'False Positive': fp,
        'False Negative': fn,
        'Accuracy': (tp + tn) / (tp + tn + fp + fn),
        'Precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
        'Recall': tp / (tp + fn) if (tp + fn) > 0 else 0,
        'F1 Score': report['macro avg']['f1-score'],
        'AUC-ROC': roc_auc,
        'AUC-PR': pr_auc,
        'Optimal Confidence Threshold': optimal_threshold
    }

    summary_df = pd.DataFrame({k: [v] for k, v in summary.items()}).T
    summary_df.columns = ['Value']
    print("\nPerformance Summary:")
    print(summary_df)
    summary_df.to_csv('performance_summary.csv')

    # Return summary metrics dictionary
    return summary

# MAIN EXECUTION

# 1. Load the model
model_path = '/content/drive/MyDrive/model testing/92ACC.pth'
model = load_model(model_path)

if model is None:
    print("Failed to load model. Exiting.")
else:
    # 2. Define test directories (using your provided paths)
    real_dir = '/content/drive/MyDrive/model testing/archive1.zip (Unzipped Files)/real_vs_fake/real-vs-fake/test/real'
    fake_dir = '/content/drive/MyDrive/model testing/archive1.zip (Unzipped Files)/real_vs_fake/real-vs-fake/test/fake'

    # 3. Run batch testing
    true_labels, pred_labels, confidences, file_paths, results_df = batch_test_videos(real_dir, fake_dir, model)

    # 4. Calculate and display metrics
    if true_labels:
        print("\nCalculating performance metrics...")
        metrics = calculate_metrics(true_labels, pred_labels, confidences, file_paths, results_df)

        print("\nTesting complete! Visualizations and reports have been saved.")
    else:
        print("No valid test data was processed.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Initializing TPU...
Using device: xla:0
Loading model from: /content/drive/MyDrive/model testing/92ACC.pth
Model loaded successfully!
Starting batch testing from directories:
Real: /content/drive/MyDrive/model testing/archive1.zip (Unzipped Files)/real_vs_fake/real-vs-fake/test/real
Fake: /content/drive/MyDrive/model testing/archive1.zip (Unzipped Files)/real_vs_fake/real-vs-fake/test/fake
Processing REAL videos...


0it [00:00, ?it/s]


Processing FAKE videos...


100%|██████████| 1971/1971 [00:00<00:00, 2557059.44it/s]

Detailed results saved to model_evaluation_results.csv
No valid test data was processed.
